In [ ]:
"""
  GitHubよりサンプルをダウンロード
"""
!git clone https://github.com/caribou68/ConvertMovieStyle.git
!mkdir ConvertMovieStyle/converted_images ConvertMovieStyle/diveded_images ConvertMovieStyle/out

#動画分解

In [ ]:
!pip install ffmpeg-python
import ffmpeg

In [ ]:
!ffmpeg -i "//content/ConvertMovieStyle/content_movie/sample.mp4" -progress - -r 60 "//content/ConvertMovieStyle/diveded_images/%06d.png"

#画風変換

In [ ]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
import numpy as np
import PIL.Image

!pip install tqdm
import tqdm

import tensorflow_hub as hub

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [ ]:
content_dir = '//content/ConvertMovieStyle/diveded_images/'
img_num = sum(os.path.isfile(os.path.join(content_dir,name)) for name in os.listdir(content_dir))
style_path = "//content/ConvertMovieStyle/style_images/sample2.jpg"
style_image = load_img(style_path)

In [ ]:
for i in tqdm.tqdm(range(1, img_num)):
  content_path = content_dir + str(i).zfill(6) + ".png"

  content_image = load_img(content_path)

  hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
  converted_img = tensor_to_image(stylized_image)
  converted_img.save("//content/ConvertMovieStyle/converted_images/" + str(i).zfill(6) + ".png")

#動画合成

In [ ]:
!ffmpeg -framerate 60 -i "//content/ConvertMovieStyle/converted_images/%06d.png" "//content/ConvertMovieStyle/out/out.mp4" -y